In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('WikiQASent.pos.ans.tsv', sep='\t')

In [ ]:
df.head()

In [8]:
data_set = df[['QuestionID', 'Question']]

In [22]:
data_set.head(12)

,QuestionID,Question
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US
1,Q1,how are glacier caves formed?
2,Q4,how a water pump works
3,Q11,"how big is bmc software in houston, tx"
4,Q11,"how big is bmc software in houston, tx"
5,Q16,how much is 1 tablespoon of water
6,Q16,how much is 1 tablespoon of water
7,Q16,how much is 1 tablespoon of water
8,Q17,how much are the harry potter movies worth
9,Q18,how a rocket engine works


## NLP Preprocessing

1. Tokenization
2. Lower + stem
3. exclude punctuation characters
4. Bag of Words

In [41]:
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import string

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#For first time
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Tokenization & remove punctuation

In [38]:
def tokenize(sentence):
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    return nltk.word_tokenize(sentence)

### Stem

In [27]:
stemmer = PorterStemmer()
def stem(word):
    return stemmer.stem(word.lower())

### Bag of Words

In [18]:
def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    
    for idex, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idex] = 1.0
    return bag

## Training a model

#### AllWords & QTag & InOutput

In [49]:
all_words = []
Qtags = []
XY = []

for question in data_set.values:
    Qtags.append(question[0])
    all_words.extend(tokenize(question[1]))
    XY.append((tokenize(question[1]), question[0]))

In [57]:
all_words = [stem(w) for w in all_words]

In [59]:
all_words = sorted(set(all_words))

#### X-Train Y-Train

In [73]:
x_train = []
y_train = []

In [75]:
for sentence, tag in XY:
    bag = bag_of_words(sentence, all_words)
    x_train.append(bag)
    
    label = Qtags.index(tag)
    y_train.append(label)

In [77]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [80]:
print(len(y_train) == len(x_train))

True


#### Chat-Data-Set Class

In [84]:
class ChatDataSet(Dataset):
    def __init__(self):
        self.n_samples = len(y_train)
        self.x_data = x_train
        self.y_data = y_train
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

#### Nerual Netword Class

In [85]:
class NerualNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NerualNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        return out
        
        

#### Training

In [ ]:
batch_size = 8
hidden_size = 8
output_size = len(Qtags)  # Number of different class
input_size = len(x_train[0])  # Number of the bag of words len(all_words)
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataSet()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NerualNet(input_size, hidden_size, output_size).to(device)


# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch_ in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        # Forward
        outputs = model(words)
        labels = labels.type(torch.LongTensor)
        loss = criterion(outputs, labels)

        # Backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch_ + 1) % 100 == 0:
        print(f"Epoch {epoch_ + 1}/{num_epochs}, loss={loss.item():.4f}")

print(f"Final loss, loss={loss.item():.4f}")

data_ = {"model_state": model.state_dict, "input_size": input_size, "output_size": output_size, "hidden_size": hidden_size, "all_words": all_words, "tags": Qtags}

FILE = "data.pth"
torch.save(data_, FILE)


print(f"training complete. file saved to {FILE}")


In [ ]:
"""
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
True
Epoch 100/1000, loss=0.0240
Epoch 200/1000, loss=0.0000
Epoch 300/1000, loss=0.0002
Epoch 400/1000, loss=0.0000
Epoch 500/1000, loss=0.0000
Epoch 600/1000, loss=0.0000
Epoch 700/1000, loss=0.0000
Epoch 800/1000, loss=0.0000
Epoch 900/1000, loss=0.0000
Epoch 1000/1000, loss=0.0000
Final loss, loss=0.0000
"""